In [1]:
import torch
import sys
import os
import json
import time
import numpy as np

sys.path.append("..")
from witsingleVis.SingleVisualizationModel import VisModel
from witsingleVis.data import NormalDataProvider
from witsingleVis.eval.evaluator import Evaluator
from witsingleVis.projector import TimeVisProjector

2023-03-24 14:52:19.989138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 14:52:20.123843: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-24 14:52:20.663057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-24 14:52:20.663099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
VIS_METHOD = "TimeVis" 

In [3]:
CONTENT_PATH ="/home/xianglin/data/BadNet_MNIST_noise"
GPU_ID = "0"

In [4]:
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [5]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]
# GPU_ID = config["GPU"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]
EPOCH_NAME = config["EPOCH_NAME"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]

VIS_MODEL_NAME = VISUALIZATION_PARAMETER["VIS_MODEL_NAME"]
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

# Define data_provider
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, epoch_name=EPOCH_NAME, device=DEVICE, classes=CLASSES,verbose=1)

Finish initialization...


In [6]:
model = VisModel(ENCODER_DIMS, DECODER_DIMS)
projector = TimeVisProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)

In [7]:
from witsingleVis.training_dynamics import TD
td = TD(data_provider, projector)

In [9]:
losses = td.loss_dynamics()

100%|██████████| 300/300 [00:00<00:00, 430.19it/s]


In [12]:
import json
with open("/home/xianglin/data/BadNet_MNIST_noise/noisy_label.json", "r") as f: 
    noise_labels = np.array(json.load(f))
with open("/home/xianglin/data/BadNet_MNIST_noise/clean_label.json", "r") as f: 
    clean_labels = np.array(json.load(f))

idxs = np.argwhere(noise_labels!=clean_labels).squeeze()

In [13]:
td.show_ground_truth(loss, idxs)